### 导入订单

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
from prophet import Prophet
import os
from pyecharts.charts import Line
from pyecharts import options as opts
from pyecharts.globals import ThemeType

c:\Users\ASUS\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### 读取订单数据

In [2]:
# 设置数据文件路径
data_dir = "../data/"

# 动态获取文件夹中的所有Excel文件
excel_files = [f for f in os.listdir(data_dir) if f.endswith(('.xlsx', '.xls'))]

# 检查是否有Excel文件
if not excel_files:
    raise FileNotFoundError(f"在目录 {data_dir} 中没有找到Excel文件。")

# 读取所有Excel文件
dfs = []
for file in excel_files:
    file_path = os.path.join(data_dir, file)
    # 检查文件是否存在
    if not os.path.exists(file_path):
        print(f"文件 {file_path} 不存在，请检查路径。")
        continue
    # 读取Excel文件
    df = pd.read_excel(file_path, engine='openpyxl')
    dfs.append(df)

# 合并数据
df = pd.concat(dfs, ignore_index=True)

### 数据预处理

In [3]:
# 数据预处理
df['支付时间'] = pd.to_datetime(df['支付时间'])
df['商品'] = df['商品'].str.split('+').str[0].str.strip()
df['支付时间'] = df['支付时间'].dt.floor('h')

# 按小时聚合数据
hourly_data = df.groupby(['支付时间', '商品'])['订单金额'].count().reset_index()
hourly_data.rename(columns={'订单金额': '订单数量'}, inplace=True)

# 创建完整的时间序列索引
start_date = hourly_data['支付时间'].min()
end_date = hourly_data['支付时间'].max()
all_hours = pd.date_range(start=start_date, end=end_date, freq='h')
all_commodities = hourly_data['商品'].unique()

# 创建完整的时间-商品组合
multi_index = pd.MultiIndex.from_product([all_hours, all_commodities], names=['支付时间', '商品'])
complete_data = pd.DataFrame(index=multi_index).reset_index()
complete_data = complete_data.merge(hourly_data, on=['支付时间', '商品'], how='left').fillna(0)

# 创建商品标签编码器
le = LabelEncoder()
complete_data['商品_encoded'] = le.fit_transform(complete_data['商品'])

### 此处模型的预测，可以调整以下参数
### periods=此处写预测的小时eg:72
### tail(此处写预测的小时)
## 下面 填写预测的小时时间

In [4]:
hour=12

In [5]:
# 预测未来hour小时
future_predictions = []
for commodity in all_commodities:
    # 筛选出当前商品的数据
    commodity_df = complete_data[complete_data['商品'] == commodity][['支付时间', '订单数量']]
    # 重命名列以符合Prophet的要求
    commodity_df = commodity_df.rename(columns={'支付时间': 'ds', '订单数量': 'y'})
    # 创建Prophet模型并拟合数据
    model = Prophet(changepoint_prior_scale=0.1, seasonality_mode='multiplicative')
    model.fit(commodity_df)
    # 创建未来72小时的时间点
    future = model.make_future_dataframe(periods=hour, freq='h')
    # 进行预测
    forecast = model.predict(future)
    # 提取预测结果
    forecast_df = forecast[['ds', 'yhat']].tail(hour)
    # 将商品名称和预测结果添加到列表中
    forecast_df['商品'] = commodity
    future_predictions.append(forecast_df)

22:00:37 - cmdstanpy - INFO - Chain [1] start processing
22:00:38 - cmdstanpy - INFO - Chain [1] done processing
22:00:38 - cmdstanpy - INFO - Chain [1] start processing
22:00:38 - cmdstanpy - INFO - Chain [1] done processing
22:00:39 - cmdstanpy - INFO - Chain [1] start processing
22:00:39 - cmdstanpy - INFO - Chain [1] done processing
22:00:39 - cmdstanpy - INFO - Chain [1] start processing
22:00:40 - cmdstanpy - INFO - Chain [1] done processing
22:00:40 - cmdstanpy - INFO - Chain [1] start processing
22:00:40 - cmdstanpy - INFO - Chain [1] done processing
22:00:40 - cmdstanpy - INFO - Chain [1] start processing
22:00:41 - cmdstanpy - INFO - Chain [1] done processing
22:00:41 - cmdstanpy - INFO - Chain [1] start processing
22:00:41 - cmdstanpy - INFO - Chain [1] done processing
22:00:41 - cmdstanpy - INFO - Chain [1] start processing
22:00:42 - cmdstanpy - INFO - Chain [1] done processing
22:00:42 - cmdstanpy - INFO - Chain [1] start processing
22:00:42 - cmdstanpy - INFO - Chain [1]

In [7]:
# 创建预测结果DataFrame
future_df = pd.concat(future_predictions, ignore_index=True)
future_df.columns = ['时间', '预测数量', '商品']
future_df = future_df[['时间', '商品', '预测数量']]

# 将预测数量四舍五入并转换为整数
future_df['预测数量'] = future_df['预测数量'].apply(lambda x: max(0, int(round(x))))

# 打印预测结果
print("未来{}小时的预测结果:".format(hour))
print(future_df)

# 可视化预测结果（使用pyecharts）
# 创建时间序列作为 x 轴
time_axis = future_df['时间'].dt.strftime('%m-%d %H:%M').unique().tolist()

# 创建折线图
line = Line(init_opts=opts.InitOpts(theme=ThemeType.LIGHT, width="1200px", height="700px"))

line.add_xaxis(time_axis)

# 为每个商品添加折线系列
for commodity in future_df['商品'].unique():
    commodity_data = future_df[future_df['商品'] == commodity]
    line.add_yaxis(
        series_name=commodity,
        y_axis=commodity_data['预测数量'].tolist(),
        symbol="circle",
        symbol_size=8,
        linestyle_opts=opts.LineStyleOpts(width=3),
        label_opts=opts.LabelOpts(is_show=False),
        is_smooth=True
    )

# 设置图表配置
line.set_global_opts(
    title_opts=opts.TitleOpts(title='未来{}小时商品订单预测'.format(hour), pos_left='center', title_textstyle_opts=opts.TextStyleOpts(font_size=20)),
    xaxis_opts=opts.AxisOpts(
        type_="category",
        axislabel_opts=opts.LabelOpts(rotate=45, font_size=12, interval="auto"),
        splitline_opts=opts.SplitLineOpts(is_show=True),
        axispointer_opts=opts.AxisPointerOpts(is_show=True, link=[{"xAxisIndex": "all"}])
    ),
    yaxis_opts=opts.AxisOpts(
        name='预测订单数量',
        name_textstyle_opts=opts.TextStyleOpts(font_size=14),
        splitline_opts=opts.SplitLineOpts(is_show=True)
    ),
    legend_opts=opts.LegendOpts(
        type_='scroll',
        pos_top='10%',
        pos_right='5%',
        orient='vertical',
        textstyle_opts=opts.TextStyleOpts(font_size=12)
    ),
    tooltip_opts=opts.TooltipOpts(trigger="axis", axis_pointer_type="cross")
)

# 设置系列配置
line.set_series_opts(
    markpoint_opts=opts.MarkPointOpts(
        data=[
            opts.MarkPointItem(type_='max', name='最大值'),
            opts.MarkPointItem(type_='min', name='最小值')
        ]
    ),
    markline_opts=opts.MarkLineOpts(
        data=[opts.MarkLineItem(type_='average', name='平均值')]
    )
)



# 同时生成 HTML 文件
line.render('forecast.html')

# 统计每个商品的总订单数量
total_orders_per_commodity = future_df.groupby('商品')['预测数量'].sum()

# 统计所有商品的总订单数量
total_orders_all_commodities = future_df['预测数量'].sum()

# 统计时间范围
start_time = future_df['时间'].min()
end_time = future_df['时间'].max()

# 打印统计信息
print(f"预测时间范围：从 {start_time} 到 {end_time}")
print("每个商品的总订单数量：")
print(total_orders_per_commodity)
print(f"\n所有商品的总订单数量：{total_orders_all_commodities}")

# 在 Jupyter Notebook 中显示图表
line.render_notebook()

未来12小时的预测结果:
                     时间             商品  预测数量
0   2025-06-15 23:00:00  品种1荔枝绿1斤（妃子笑）     0
1   2025-06-16 00:00:00  品种1荔枝绿1斤（妃子笑）     0
2   2025-06-16 01:00:00  品种1荔枝绿1斤（妃子笑）     0
3   2025-06-16 02:00:00  品种1荔枝绿1斤（妃子笑）     0
4   2025-06-16 03:00:00  品种1荔枝绿1斤（妃子笑）     0
..                  ...            ...   ...
103 2025-06-16 06:00:00   品种2荔枝红1斤（桂味）     2
104 2025-06-16 07:00:00   品种2荔枝红1斤（桂味）     2
105 2025-06-16 08:00:00   品种2荔枝红1斤（桂味）     2
106 2025-06-16 09:00:00   品种2荔枝红1斤（桂味）     2
107 2025-06-16 10:00:00   品种2荔枝红1斤（桂味）     2

[108 rows x 3 columns]
预测时间范围：从 2025-06-15 23:00:00 到 2025-06-16 10:00:00
每个商品的总订单数量：
商品
品种1荔枝绿1斤          0
品种1荔枝绿1斤（妃子笑）     0
品种2荔枝红1斤          0
品种2荔枝红1斤（桂味）     20
品种2荔枝红1斤（白糖罂）     0
山竹1斤              1
杨梅1斤（袋子装）         0
桃子1斤              0
荔枝各半              0
Name: 预测数量, dtype: int64

所有商品的总订单数量：21
